# Mulai Disini

In [1]:
!nvidia-smi

Tue Nov 28 09:00:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 15833 (delta 157), reused 222 (delta 140), pack-reused 15539
Receiving objects: 100% (15833/15833), 14.75 MiB | 16.86 MiB/s, done.
Resolving deltas: 100% (10587/10587), done.


In [4]:
%cd /content/darknet/

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile


/content/darknet


In [5]:
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

In [6]:
!pwd

/content/darknet


# Pre-Trained configure layers to adjust data needed

In [7]:
#make a coppp yolo.cfg

!cp cfg/yolov3.cfg  cfg/yolov3_training.cfg

In [8]:
!sed -i '3 s@batch=1@batch=8@' cfg/yolov3_training.cfg
!sed -i '4 s@subdivisions=1@subdivisions=16@' cfg/yolov3_training.cfg
!sed -i '20 s@max_batches = 500200@max_batches=2000@' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
!echo -e 'motorbike' > /content/darknet/data/obj.names
!echo -e 'classes = 1 \ntrain = /content/darknet/data/train.txt\nvalid = /content/darknet/data/test.txt\nnames = /content/darknet/data/obj.names\nbackup = /content/drive/MyDrive/yoloV3' >  /content/darknet/data/obj.data

# OPTIONAL (jika belum ada di drive)

In [12]:
!cp /content/darknet/cfg/yolov3_training.cfg /content/drive/MyDrive/yoloV3/yolov3_testing.cfg
!cp /content/darknet/data/obj.names /content/drive/MyDrive/yoloV3/classes.txt

In [13]:
!mkdir /content/drive/MyDrive/yoloV3/data/
!mkdir /content/drive/MyDrive/yoloV3/data/obj/


In [16]:
%cd /

/


In [17]:
import zipfile

xip = '/content/drive/MyDrive/yoloV3/images.zip'
with zipfile.ZipFile(xip , 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/yoloV3/data/obj/')


In [18]:
import glob
# pada folder ./data/ akan membuat file train.txt yang digunakan menulis di setiap lisd
lisd = glob.glob('/content/drive/MyDrive/yoloV3/data/obj/*.jpg')
with open('/content/darknet/data/train.txt', 'w') as f:
  f.write('\n'.join(lisd))


# LOAD PRE-TRAINED WEIGHTS FOR CONVOLUTIONAL LAYERS FILE

In [20]:
%cd /content/darknet/
!pwd

/content/darknet
/content/darknet


In [21]:
!wget  https://pjreddie.com/media/files/darknet53.conv.74

--2023-11-28 09:11:09--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  22.2MB/s    in 8.5s    

2023-11-28 09:11:19 (18.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [22]:
%cd ../../

/


In [ ]:
# !python darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# !cd darknet && ./darknet detector /content/darknet/data/obj.data  /content/darknet/cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
!cd darknet && ./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show



Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.751513), count: 77, class_loss = 5.296874, iou_loss = 2.108517, total_loss = 7.405391 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.704269), count: 46, class_loss = 2.439519, iou_loss = 1.751992, total_loss = 4.191511 
 total_bbox = 94557, rewritten_bbox = 0.353226 % 
1335/2000: loss=4.0 hours left=0.1
 1335: 4.000530, 2.923678 avg loss, 0.001000 rate, 0.918723 seconds, 10680 images, 0.127004 hours left
Loaded: 0.000088 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.764404), count: 3, class_loss = 0.239096, iou_loss = 0.061884, total_loss = 0.300980 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.762138), count: 41, class_loss = 1.711646, iou_loss = 0.959517, total_loss = 2.671163 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00)

In [43]:
%cd /
!pwd

/
/


In [37]:
!cd /content/darknet && ./darknet detector test data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/yoloV3/yolov3_training_final.weights


 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5, 

In [39]:
import os

# Folder containing images
image_folder = '/content/drive/MyDrive/yoloV3/test'

# Darknet command template
darknet_command_template = (
    "/content/darknet/darknet detector test data/obj.data "
    "cfg/yolov3_training.cfg /content/drive/MyDrive/yoloV3/yolov3_training_final.weights {}"
)

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        # Full path to the image
        image_path = os.path.join(image_folder, image_file)

        # Build the Darknet command with the image path
        darknet_command = darknet_command_template.format(image_path)

        # Run Darknet command and capture output
        darknet_output = os.popen(darknet_command).read()

        # Process Darknet output
        lines = darknet_output.split('\n')
        motorbike_count = sum(1 for line in lines if line.startswith('motorbike'))

        # Print or use the results
        print(f"File: {image_file}, Motorbike Count: {motorbike_count}")


File: parking1_11-12-2023_17_00_02.jpg, Motorbike Count: 0
File: parking1_11-12-2023_17_10_01.jpg, Motorbike Count: 0
File: parking1_11-12-2023_17_20_02.jpg, Motorbike Count: 0
File: parking1_11-13-2023_04_50_02.jpg, Motorbike Count: 0
File: parking1_11-13-2023_05_40_02.jpg, Motorbike Count: 0
File: parking1_11-13-2023_20_10_02.jpg, Motorbike Count: 0
File: parking1_11-13-2023_20_20_02.jpg, Motorbike Count: 0
File: parking1_11-13-2023_16_00_04.jpg, Motorbike Count: 0
File: parking1_11-14-2023_12_40_09.jpg, Motorbike Count: 0
File: parking1_11-14-2023_14_50_05.jpg, Motorbike Count: 0


In [47]:
%cd /content/

/content
